<a href="https://colab.research.google.com/github/GSPriya13/Twibuzz/blob/main/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tweets Extraction from API

In [ ]:
import requests
import time
import json

# Replace with your actual Bearer Token
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKImxwEAAAAAVMpCA4X5M6IsgQ0UtvGaw%2FVgcTM%3DkdUyLVzxr4WivluvE2FwTz0dApZrpnMmjQm3KHxvEWKXYyRBDr"

# Define the endpoint and headers
BASE_URL = "https://api.twitter.com/2/tweets/search/recent"
HEADERS = {
    "Authorization": f"Bearer {BEARER_TOKEN}"
}

# Parameters for the API request
PARAMS = {
    "query": "Christmas lang:en",  # Search query
    "max_results": 100,            # Fetch up to 100 tweets at a time
    "tweet.fields": "text,created_at,public_metrics,geo,entities",  # Tweet fields
    "user.fields": "username,location",                            # User fields
    "expansions": "author_id,geo.place_id",                        # Related data
}

# Initialize variables for data collection
tweets = []  # To store all fetched tweets
pagination_token = None  # Used for pagination

try:
    while True:
        # Add pagination token if it exists
        if pagination_token:
            PARAMS["next_token"] = pagination_token

        # Make the request to the API
        response = requests.get(BASE_URL, headers=HEADERS, params=PARAMS)

        # Raise exception for HTTP errors
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Extract tweets if available
        if "data" in data:
            # Combine tweet data with additional user/geo information
            includes = data.get("includes", {})
            users = {user["id"]: user for user in includes.get("users", [])}
            places = {place["id"]: place for place in includes.get("places", [])}

            # Merge tweet data with user and geo details
            for tweet in data["data"]:
                tweet["user"] = users.get(tweet["author_id"], {})
                tweet["place"] = places.get(tweet.get("geo", {}).get("place_id"), {})
                tweets.append(tweet)

        # Check if there's a next_token for pagination
        pagination_token = data.get("meta", {}).get("next_token")

        # Break the loop if no next_token is available
        if not pagination_token:
            print("No more data to fetch.")
            break

        # Respect API rate limits (adjust as needed)
        time.sleep(1)

    print(f"Total tweets fetched: {len(tweets)}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

# Save data to a JSON file
with open("tweets.json", "w") as file:
    json.dump(tweets, file, indent=4)

print("Data saved to tweets.json")

In [ ]:
import json
import pandas as pd

# Load the JSON data
with open("tweets.json", "r") as file:
    tweets = json.load(file)

# Convert to a DataFrame for easier manipulation
df = pd.json_normalize(tweets)
print(df.head())

                                                text  \
0  RT @_s_l_a_t_e_: 🎄 Happy Holidays, SL8 Family!...   
1  ‌How did you spend your Christmas? 🤭\nWere you...   
2  RT @LeeAndersonMP_: How dare people accuse us ...   
3  New Year Lucky Roulette\nA limited-time New Ye...   
4  RT @XenoLola: Rate my my body in the comments ...   

                 created_at edit_history_tweet_ids                   id  \
0  2024-12-28T11:09:37.000Z  [1872963109448712298]  1872963109448712298   
1  2024-12-28T11:09:37.000Z  [1872963109180326012]  1872963109180326012   
2  2024-12-28T11:09:37.000Z  [1872963108949639541]  1872963108949639541   
3  2024-12-28T11:09:37.000Z  [1872963108001726836]  1872963108001726836   
4  2024-12-28T11:09:37.000Z  [1872963106651160751]  1872963106651160751   

             author_id                                      entities.urls  \
0  1872769255747534848  [{'start': 82, 'end': 105, 'url': 'https://t.c...   
1  1696810902618550272                                    

Explore the Data

In [ ]:
# View basic information
print(df.info())

# Check for missing values
print(df.isnull().sum())

# Display a sample tweet
print(df.iloc[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   text                             100 non-null    object
 1   created_at                       100 non-null    object
 2   edit_history_tweet_ids           100 non-null    object
 3   id                               100 non-null    object
 4   author_id                        100 non-null    object
 5   entities.urls                    35 non-null     object
 6   entities.mentions                94 non-null     object
 7   public_metrics.retweet_count     100 non-null    int64 
 8   public_metrics.reply_count       100 non-null    int64 
 9   public_metrics.like_count        100 non-null    int64 
 10  public_metrics.quote_count       100 non-null    int64 
 11  public_metrics.bookmark_count    100 non-null    int64 
 12  public_metrics.impression_count  100 